In [ ]:
import sys
!{sys.executable} -m pip install pandas

import os
import sqlite3
import pandas as pd


In [3]:
LINE_DELIMITER = '*' * 50


def generate_data(conn):
    c = conn.cursor()
    # create the 'employees' table
    c.execute('''CREATE TABLE IF NOT EXISTS employees
                 (id INTEGER PRIMARY KEY, name TEXT, department_id INTEGER, salary INTEGER)''')
    # create the 'departments' table
    c.execute('''CREATE TABLE IF NOT EXISTS departments
                 (id INTEGER PRIMARY KEY, department_name TEXT)''')
    # insert data into the 'departments' table
    departments_data = [
        (1, "Human Resources"),
        (2, "Sales"),
        (3, "Engineering")
    ]
    c.executemany('''INSERT INTO departments (id, department_name) VALUES (?, ?)''', departments_data)
    # insert data into the 'employees' table
    employees_data = [
        (1, "emp1", 1, 30000),
        (2, "emp2", 2, 40000),
        (3, "emp3", 2, 54000),
        (4, "emp4", 2, 56000),
        (5, "emp5", 3, 80000),
        (6, "emp6", 3, 555555), # Highest paid employee (Engineer)
        (7, "emp7", 2, 190000)  # Sales employee with outlier salary
    ]
    c.executemany('''INSERT INTO employees (id, name, department_id, salary) VALUES (?, ?, ?, ?)''', employees_data)
    # save into database
    conn.commit()

In [5]:
# [1] create a connection 
db_file = "company.db"
if os.path.exists(db_file):
  os.remove(db_file)  # just for testing, delete db every time
conn = sqlite3.connect(db_file)  # will create db if not exists
generate_data(conn)  # generate test data

# [2] fetch employees from Sales department where salary > 50000
query = """SELECT e.name, e.salary, d.department_name
            FROM employees e
            JOIN departments d ON e.department_id = d.id
            -- WHERE d.department_name = 'Sales' AND e.salary > 50000"""
joint_employees_departments = pd.read_sql(query, conn)
sales_employees_above_50k = joint_employees_departments.query("department_name == 'Sales' and salary > 50000")
print('[2] Employees from Sales with salary above 50000\n', sales_employees_above_50k.to_string(index=False))
print(LINE_DELIMITER)

# [3] using pandas interface to calculate average and median salary
print(f"""[3] Calculates the average and median salary of these employees
\tAverage salary:\t{sales_employees_above_50k['salary'].mean()}
\tMedian salary:\t{sales_employees_above_50k['salary'].median()}""")
print(LINE_DELIMITER)

# [4] group data by department and print the highest earner in each department
print("[4] Highest earner for each department")
for department, data in joint_employees_departments.groupby("department_name"):
    highest_earner = data.nlargest(1, "salary")
    print(highest_earner.to_string(index=False, header=False))
print(LINE_DELIMITER)

# [5] find employee with the highest salary in the entire company
highest_salary_emp_index = joint_employees_departments["salary"].idxmax()
highest_salary_employee = joint_employees_departments.loc[highest_salary_emp_index]
print(f"""[5] Employee with the highest salary in the entire company:
\t{highest_salary_employee['name']} with salary {highest_salary_employee['salary']}""")

# [6] save the results to a csv file
sales_employees_above_50k.to_csv("sales_employees_salary_above_50k.csv", index=False)
conn.close()

[2] Employees from Sales with salary above 50000
 name  salary department_name
emp3   54000           Sales
emp4   56000           Sales
emp7  190000           Sales
**************************************************
[3] Calculates the average and median salary of these employees
	Average salary:	100000.0
	Median salary:	56000.0
**************************************************
[4] Highest earner for each department
emp6 555555 Engineering
emp1 30000 Human Resources
emp7 190000 Sales
**************************************************
[5] Employee with the highest salary in the entire company:
	emp6 with salary 555555
